<a href="https://colab.research.google.com/github/Gru97/DeepLearningCourse/blob/main/session5_excercise_lenet5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import transforms


In [91]:
device='cpu'

train_dataset= datasets.CIFAR10('/content/', train=True, download=True, transform=transforms.ToTensor())
test_dataset= datasets.CIFAR10('/content/', train=False, download=True, transform=transforms.ToTensor())
train_x= train_dataset.data
train_y=train_dataset.targets

test_x= test_dataset.data
trest_y= test_dataset.targets

Files already downloaded and verified
Files already downloaded and verified


In [92]:
print(torch.Tensor(train_x).shape)
torch.Tensor(train_x).permute(0, 3, 1, 2).shape  # Rearrange to (N, C, H, W).shape

torch.Size([50000, 32, 32, 3])


torch.Size([50000, 3, 32, 32])

In [93]:
print(train_x.shape) #(50000, 32, 32, 3)
#print(train_y.shape) #[50000])
print(train_x[0].shape) #(32, 32, 3) image
#print(train_y.unique()) #[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
train_dataset.classes

(50000, 32, 32, 3)
(32, 32, 3)


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [141]:
from torch import nn
class Lenet5(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1= nn.Conv2d(3,6,5)  # 1= grey, 6= number of filters 5= size of filter (32-28+1=5)
    self.pool1= nn.MaxPool2d(2) # 28 has turned 14 so MaxPool(2)
    self.conv2= nn.Conv2d(6,16,5) # 6 previous filters result in 6 channel features maps, 16 filters in this step, 10-14+1=5 size of filter
    self.pool2=nn.MaxPool2d(2)
    self.flatten=nn.Flatten()
    self.fc1=nn.Linear(16*5*5,120)
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,10)
    self.relu= nn.ReLU()

  def featurize(self, x):
    x=self.conv1(x)
    x=self.pool1(x)
    x=self.relu(x)
    x=self.conv2(x)
    x=self.pool2(x)
    x=self.relu(x)
    x=self.flatten(x)
    return x
  def classifier(self, x):
    x= self.fc1(x)
    x= self.relu(x)
    x= self.fc2(x)
    x= self.relu(x)
    x=self.fc3(x)
    return x
  def forward(self, x):
    x= self.featurize(x)
    x= self.classifier(x)
    return x;

In [157]:
class CustomDataset(Dataset):
  def __init__(self, data, target):
    self.data = torch.tensor(data, dtype=torch.float32)/255
    self.data= self.data.permute(0, 3, 1, 2)  # Rearrange to (N, C, H, W)
    self.target=torch.tensor(target)
  def __getitem__(self, id):
    return self.data[id], self.target[id]
  def __len__(self):
    return len(self.data)

In [158]:
batch_size=32
train_dataset= CustomDataset(train_x , train_y)
validation_dataset= CustomDataset(test_x, trest_y)
train_dataset.__len__()
train_dataset.__len__()/batch_size

1562.5

In [159]:
train_dataloader= DataLoader(train_dataset,batch_size, shuffle=True)
test_dataloader= DataLoader(validation_dataset, batch_size, shuffle=True)
len(train_dataloader)  #len/batch_size

1563

In [166]:
def accuracy(output, targets):
  _, predictions = torch.max(output, 1)  # class labels of max possiblitity resulted from a neuron

  correct= (predictions==targets).sum().item()
  return correct/targets.size(0)

In [172]:
lr=0.1
model= Lenet5().to(device)
optimizer= torch.optim.SGD(model.parameters(), lr= lr)
loss_function= nn.CrossEntropyLoss()  # for multi class data

def train(model, dataloader, loss_function, optimizer):
  total_loss=0.0
  total_acc=0.0
  for inputs, targets in dataloader:  # len(dataloader)/batch_size = count for
    inputs, targets = inputs.to(device), targets.to(device)
    model.train()
    outputs = model(inputs)
    loss_of_a_batch = loss_function(outputs,targets)
    loss_of_a_batch.backward()
    optimizer.step()
    optimizer.zero_grad()
    total_loss += loss_of_a_batch.item()
    total_acc += accuracy(outputs,targets)
  return total_loss/len(dataloader), total_acc/len(dataloader)  #mean in every epoch

def validate(model, dataloader, loss_function):
    model.eval()
    total_loss=0.0
    total_acc=0.0
    with torch.no_grad():
      for inputs, targets in dataloader:
        inputs, targets= inputs.to(device), targets.to(device)
        outputs= model(inputs)
        loss= loss_function(outputs,targets)
        total_loss += loss.item()
        total_acc += accuracy(outputs,targets)
    return total_loss/len(dataloader), total_acc/len(dataloader)  #mean in every epoch



In [173]:
epoch=30
train_losses=[]
test_losses=[]
train_acc=[]
test_acc=[]
for i in range(epoch):
    train_loss, train_accuracy = train(model, train_dataloader, loss_function, optimizer)
    train_losses.append(train_loss)
    train_acc.append(train_accuracy)
    test_loss, test_accuracy =validate(model, test_dataloader, loss_function)
    test_losses.append(test_loss)
    test_acc.append(test_accuracy)

    print(f"Epoch {i+1}/{epoch}, train_loss: {train_loss}, test_loss: {test_loss}, train_acc: {train_accuracy}, test_acc: {test_accuracy}")


Epoch 1/30, train_loss: 1.930863979071741, test_loss: 1.729329214309351, train_acc: 0.29168666026871404, test_acc: 0.36461661341853036
Epoch 2/30, train_loss: 1.5359826193203625, test_loss: 1.4077651239812565, train_acc: 0.44629718490083176, test_acc: 0.4902156549520767
Epoch 3/30, train_loss: 1.4028550135883397, test_loss: 1.4104589979869488, train_acc: 0.4950615802943058, test_acc: 0.49490814696485624
Epoch 4/30, train_loss: 1.3155863973778634, test_loss: 1.357848939423363, train_acc: 0.5286708253358925, test_acc: 0.5160742811501597
Epoch 5/30, train_loss: 1.2399904592557358, test_loss: 1.3110644842108217, train_acc: 0.5558221369161869, test_acc: 0.5396365814696485
Epoch 6/30, train_loss: 1.1870032343739358, test_loss: 1.3767601015468756, train_acc: 0.5787547984644914, test_acc: 0.5296525559105432
Epoch 7/30, train_loss: 1.134631172289699, test_loss: 1.2873598579019785, train_acc: 0.5965690978886756, test_acc: 0.5470247603833865
Epoch 8/30, train_loss: 1.0939206736101528, test_loss: 